# Portafolios de Inversión
## Tarea 6

Presenta : 

- José Armando Melchor Soto

- Paula Ines Pelayo Morales 

---
### Librerías 



In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize 

---

#### Indicaciones 

- Punto a) Implementen tres funciones que optimicen portafolios utilizando: mínima semivarianza, ratio omega y semivarianza objetivo, empleando la función `scipy.optimize.minimize`. (40%)
 

- Punto b) Construyan un portafolio compuesto por 5 activos financieros y optimícenlo con los tres métodos. Utilicen las funciones desarrolladas en el punto a) para realizar la optimización. Recuerden que, para el portafolio con semivarianza objetivo, deberán seleccionar un benchmark adecuado y consistente con los activos elegidos. Como respuesta a este punto, se esperan las ponderaciones eficientes y el valor óptimo de la función objetivo para cada método de Asset Allocation. (40%)
 

- Punto c) Escriban una breve conclusión sobre las ventajas y desventajas que observan entre los métodos TMP y TPMP. (20%)

---

#### Funciones 

#### Minimizar Semi-Varianza

**Minimizar semivarianza**

$$min_w \hspace{0.5cm} \sigma_d^2 = w^T S w$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$



##### Mínima Semi-Varianza  (Montecarlo)

In [2]:
def semi_var (rets:pd.DataFrame, corr:pd.DataFrame, n_simulaciones:int):
    
    rets_below_zero=rets[rets < 0].fillna(0)
    
    downside_risk=rets_below_zero.std()

    downside_risk=np.array(downside_risk)
    
    semivar_matrix = downside_risk.reshape(len(rets.columns), 1) @ downside_risk.reshape(1, len(rets.columns)) * corr
    
    n_assets = len(rets.keys())
    
    w = np.random.dirichlet(np.ones(n_assets), n_simulaciones)

    semivar_list=[w.T @ semivar_matrix @ w for w in w ]
    
    w_semivariance=w[np.argmin(semivar_list)]

    return dict(zip(rets.columns, w_semivariance))    

##### Mínima Semi-Varianza  (Scipy Library)

In [3]:
def scipy_semivar(rets: pd.DataFrame, corr: pd.DataFrame):

    rets_below_zero=rets[rets < 0].fillna(0)
    
    downside_risk=rets_below_zero.std()

    downside_risk=np.array(downside_risk)
    
    semivar_matrix = downside_risk.reshape(len(rets.columns), 1) @ downside_risk.reshape(1, len(rets.columns)) * corr

    n = len(rets.columns)
    w_inicial = np.ones(n) / n
    bounds = [(0, 1)] * n
    tol = 1e-9

    semivar = lambda w: (w.T @ semivar_matrix @ w)

    constraints = {'fun': lambda w: np.sum(w) - 1, 'type': 'eq'}

    minima_semi_var = minimize(
        fun=semivar,
        x0=w_inicial,
        bounds=bounds,
        constraints=constraints,
        tol=tol)

    w_semi_var = minima_semi_var.x
    return dict(zip(rets.columns, w_semi_var))


---

#### Ratio Omega 

**Calcular Omega del portafolio y maximizarla cambiando ponderaciones.**

$$max_w \hspace{0.5cm} \Omega_p = \sum_{i=1}^{n} \Omega_i * w_i$$
    
$$s.a. \hspace{0.5cm} \sum_{i=1}^n w_i = 1$$
 
$$\hspace{0.8cm} w_i > 0 $$

##### Ratio Omega (Montecarlo)

In [4]:
def Omega (rets:pd.DataFrame ,n_simulaciones:int):

    rets_below_zero=rets[rets < 0].fillna(0)
    rets_above_zero=rets[rets > 0].fillna(0)
    upside_risk=rets_above_zero.std()
    upside_risk=np.array(upside_risk)
    downside_risk=rets_below_zero.std()
    downside_risk=np.array(downside_risk)
    
    random_weights = np.random.dirichlet(np.ones(len(rets.columns)), n_simulaciones)
    
    omega = lambda w: [sum(w * upside_risk/downside_risk) for w in random_weights]

    w_omega=random_weights[np.argmax(omega)]

    return dict(zip(rets.columns, w_omega))

##### Ratio Omega (Scipy Library)

In [5]:
def scipy_omega(rets: pd.DataFrame):
    n = len(rets.columns)
    w_inicial = np.ones(n) / n
    bounds = [(0, 1)] * n
    tol = 1e-9

    rets_below_zero = rets[rets < 0].fillna(0)
    rets_above_zero = rets[rets > 0].fillna(0)
    upside_risk=rets_above_zero.std()
    downside_risk = rets_below_zero.std()

    omega_ind = upside_risk / downside_risk

    Omega_neg = lambda w: -np.sum(w * omega_ind)

    constraints = {'fun': lambda w: np.sum(w) - 1, 'type': 'eq'}

    result = minimize(
        fun=Omega_neg,
        x0=w_inicial,
        bounds=bounds,
        constraints=constraints,
        tol=tol
    )

    w_omega = result.x
    return dict(zip(rets.columns, w_omega))


---

#### Semi-Varianza Objetivo

##### Semi-Varianza Objetivo (Montecarlo)

In [6]:
def monte_objetivo (rets:pd.DataFrame,rets_b:pd.DataFrame,  corr:pd.DataFrame, n_simulaciones:int):
    diffs=rets - rets_b.values
    rends_below_bench = diffs[diffs < 0].fillna(0)
    target_downside_risk=np.array(rends_below_bench.std())
    target_semivarmatrix=corr * (target_downside_risk.reshape(len(rets.columns), 1) @ target_downside_risk.reshape(1, len(rets.columns)))
    random_weights=np.random.dirichlet(np.ones(len(rets.columns)), n_simulaciones)
    
    objectivo = lambda w: (w.T @ target_semivarmatrix @ w for w in random_weights)
    
    w_target_semivariance=random_weights[np.argmin(objectivo)]

    return dict(zip(rets.columns, w_target_semivariance*100))


##### Semi-Varianza Objetivo (Scipy)

In [7]:
def objetivo (rets:pd.DataFrame , rets_b : pd.DataFrame ,corr:pd.DataFrame):

    n = len(rets.keys())
    w_inicial = np.ones(n)/n
    bounds = [(0,1)]*n
    tol = 1e-9
    rest = lambda w: np.sum(w) - 1
    
    diffs=rets - rets_b.values
    rends_below_bench = diffs[diffs < 0].fillna(0)
    target_downside_risk=np.array(rends_below_bench.std())
    target_semivarmatrix= corr * (target_downside_risk.reshape(len(rets.columns), 1) @ target_downside_risk.reshape(1, len(rets.columns)))

    objectivo = lambda w: (w.T @ target_semivarmatrix @ w)


    obj = minimize(
    fun=objectivo,
    x0=w_inicial,
    bounds=bounds,
    constraints={'fun': rest, 'type': 'eq'},
    tol=tol)

    w_objetivo = obj.x

    return dict(zip(rets.columns, w_objetivo))

---

#### Construcción del Portafolio 

##### Activos 

- 

-

-

-

-


##### Benchmark

- 

---

#### Importación de los datos

##### Activos 

In [8]:
prices = yf.download(['FNV.TO', 'NWG.L' ,'TSM' , 'SAP.DE'], start='2020-01-01', end='2025-06-19')['Close']
prices.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


Ticker,FNV.TO,NWG.L,SAP.DE,TSM
Date,,,,
2020-01-02,127.809723,262.509766,114.211533,54.292786
2020-01-03,127.524475,256.062469,112.769844,52.502319
2020-01-06,128.075989,257.889252,111.927307,51.896446
2020-01-07,128.722519,258.533936,112.432831,52.737431
2020-01-08,124.938217,258.211609,113.106865,53.126274


##### Benchmarck 

In [9]:
benchmark = yf.download('MGIAX', start='2020-01-01', end='2025-06-19')['Close']
benchmark.head()

[*********************100%***********************]  1 of 1 completed


Ticker,MGIAX
Date,
2020-01-02,27.914818
2020-01-03,27.755903
2020-01-06,27.749790
2020-01-07,27.737564
2020-01-08,27.694782


---

#### Construcción del Portafolio 

##### Rendimientos Diarios

In [10]:
rets = prices.pct_change().dropna()
rets.head()

C:\Users\ppmel\AppData\Local\Temp\ipykernel_1540\420489823.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices.pct_change().dropna()


Ticker,FNV.TO,NWG.L,SAP.DE,TSM
Date,,,,
2020-01-03,-0.002232,-0.024560,-0.012623,-0.032978
2020-01-06,0.004325,0.007134,-0.007471,-0.011540
2020-01-07,0.005048,0.002500,0.004517,0.016205
2020-01-08,-0.029399,-0.001247,0.005995,0.007373
2020-01-09,0.002207,-0.006658,0.019368,0.008170


##### Rendimiento promedio 

In [11]:
rets_mean = rets.mean()

##### Matriz de Correlación

In [12]:
corr = rets.corr()

#### Construcción de BenchMark

##### Rendimientos Diarios

In [13]:
rets_b = benchmark.pct_change().dropna()

##### Rendimiento Promedio 

In [14]:
rets_b_mean = rets_b.mean()

---

#### Mínima Semi-Varianza 

##### Montecarlo 

In [15]:
minima_semi_var = semi_var(rets,corr,10000)
minima_semi_var

{'FNV.TO': 0.3906026041152184,
 'NWG.L': 0.1624149970385182,
 'SAP.DE': 0.30348915546864746,
 'TSM': 0.14349324337761585}

In [16]:
sum(minima_semi_var.values()).round(4)

1.0

##### Scipy Library

In [17]:
scipy_semi_var = scipy_semivar(rets, corr)
scipy_semi_var

{'FNV.TO': 0.3974150106379554,
 'NWG.L': 0.17740005973637193,
 'SAP.DE': 0.2739871909072246,
 'TSM': 0.15119773871844802}

In [18]:
sum(scipy_semi_var.values())

1.0

---

#### Ratio Omega 

##### Montecarlo 

In [19]:
Om = Omega(rets,  10000)
Om

{'FNV.TO': 0.2169308190147642,
 'NWG.L': 0.03193938197049711,
 'SAP.DE': 0.6819498092908628,
 'TSM': 0.06917998972387596}

In [20]:
sum(Om.values())

1.0

##### Scipy Library

In [21]:
scipy_omega = scipy_omega(rets)
scipy_omega

{'FNV.TO': 0.0,
 'NWG.L': 5.377642775528102e-17,
 'SAP.DE': 4.5102810375396984e-17,
 'TSM': 1.0}

In [22]:
sum(scipy_omega.values())

1.0

---

#### Semi-Varianza Objetivo 

In [23]:
objetivo_weights = objetivo(rets, rets_b, corr)
objetivo_weights

ValueError: Unable to coerce to DataFrame, shape must be (1411, 4): given (1372, 1)

In [ ]:
sum(objetivo_weights.values())